# Neural Network: Predicting Used Car Prices

Single layer MLP with 100 units in the hidden layer. Trained over 10 epochs using an MSELoss.

Mean Squared Error 140004043.87968037
Root Mean Squared Error 11832.330450071126

In [2]:
#%matplotlib inline
#import CleaningCars as cc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

In [ ]:
df = pd.read_csv('data/vehicles.csv')

In [ ]:
df = cc.ultimateClean(df)

Remove outliers...
Remove outliers...
Cleaned outliers !
Imputed Missing Values


/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator KMeans from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


One hot encodings done!
Total Time:  4.664411330223084  minutes


In [ ]:
#Standardise features
cols_to_norm = ['year','odometer','lat','long']

df[cols_to_norm] = StandardScaler().fit_transform(df[cols_to_norm])

In [ ]:
#df.to_csv('clean_cars_df.csv')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
df = pd.read_csv('drive/MyDrive/DataMining/clean_cars_df.csv')

In [62]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [63]:
#Set aside test set for later
test_set_df = df.loc[int(df.shape[0] * 0.8):, :]
df = df.loc[:int(df.shape[0] * 0.8), :]

In [64]:
#Get target and features
y = df.pop('price').values
X = df.values

In [65]:
#Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [ ]:
#Neural network
#nn = MLPRegressor(random_state=1, hidden_layer_sizes=(100,), activation='relu').fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [20]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

     |████████████████████████████████| 143kB 18.5MB/s 


In [26]:
# automatically reload external modules if they change
%load_ext autoreload
%autoreload 2

import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchbearer import Trial
import os
import json
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
train_features = torch.Tensor(X_train)
train_targets = torch.Tensor(y_train)
val_features = torch.Tensor(X_val)
val_targets = torch.Tensor(y_val)

In [68]:
train_tensor = TensorDataset(train_features, train_targets) 
train_loader = DataLoader(dataset = train_tensor, batch_size = 256, shuffle = True)
val_tensor = TensorDataset(val_features, val_targets) 
val_loader = DataLoader(dataset = val_tensor, batch_size = 256, shuffle = True)

In [71]:
X_train.shape

(198828, 102)

In [81]:
# define baseline model
class CarsModel(nn.Module):
    def __init__(self):
        super(CarsModel, self).__init__()
        self.fc1 = nn.Linear(102, 100)
        self.fc2 = nn.Linear(100, 1) 
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [82]:
model = CarsModel()

In [86]:
# define the loss function and the optimiser
loss_function = nn.MSELoss()
optimiser = optim.Adam(model.parameters()) 

trial = Trial(model, optimiser, loss_function, metrics=['loss','accuracy']).to('cuda:0')
trial.with_generators(train_loader, val_generator=val_loader)
trial.run(epochs=1)
results = trial.evaluate(data_key=torchbearer.VALIDATION_DATA)
print()
print(results)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([172])) that is different to the input size (torch.Size([172, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([43])) that is different to the input size (torch.Size([43, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)




{'val_loss': 128807312.0, 'val_mse': 128150360.0}


In [87]:
128150360.0 ** 0.5

11320.351584646123

In [90]:
y_test = test_set_df.pop('price').values
X_test = test_set_df.values

KeyError: ignored

In [95]:
preds = model(torch.Tensor(X_test).to('cuda:0'))

In [107]:
preds_clean = preds.view(1, -1)[0].detach().to('cpu').numpy()

In [111]:
mse = ((preds_clean - y_test)**2).mean(axis=None)
print("Mean Squared Error", mse)
print("Root Mean Squared Error", mse**0.5)

Mean Squared Error 140004043.87968037
Root Mean Squared Error 11832.330450071126
